In [2]:
import numpy as np
import pandas as pd

In [33]:
fileToOpen = 'input.txt'

In [4]:
def IsolateContentName(string):
    length = len(string)
    
    if string == 'no other bags.':
        return (False, 0)
    elif string[length-4:length] == 'bags':
        return (string[2:length-5], int(string[0]))
    elif string[length-3:length] == 'bag':
        return (string[2:length-4], int(string[0]))
    elif string[length-5:length] == 'bags.':
        return (string[2:length-6], int(string[0]))
    elif string[length-4:length] == 'bag.':
        return (string[2:length-5], int(string[0]))
    else:
        print("ERROR: content pattern", string, "not recognized")
        return ('ERROR', 0)

def IsolateBagName(string):
    length = len(string)
    return string[:length-5]

def AppendToColumnList(columnList, bag):
    if bag != False:
        if bag in columnList:
            return columnList
        else:
            return columnList.append(bag)
        
def AppendToNonRepeatingList(lst, string):
    if string in lst:
        return lst
    else:
        return lst.append(string)

## READ THE DATA

In [34]:
## First run: we get all bag names

columnList = []

with open(fileToOpen) as openFileObject:
    for line in openFileObject:
        bag, content = line.rstrip().split(" contain ")
        AppendToColumnList(columnList, IsolateBagName(bag))
#        for element in content.split(", "):
#            AppendToColumnList(columnList, IsolateContentName(element)[0])

#print(columnList)
print(len(columnList))

594


In [35]:
## Second run: we actually fill our dataframe
bagData = pd.DataFrame(columns=columnList)
dictionary = {}
progress = 0

with open(fileToOpen) as openFileObject:
    for line in openFileObject:
        progress += 1
        if progress % 20 == 0:
            print("Processing bag", progress, "of", len(columnList))
        bag, content = line.rstrip().split(" contain ")
        for element in content.split(", "):
            dictionary[IsolateContentName(element)[0]] = IsolateContentName(element)[1]
        
        bagData.loc[IsolateBagName(bag)] = dictionary
        dictionary.clear()

bagData = bagData.fillna(0)
bagData.head()

Processing bag 20 of 594
Processing bag 40 of 594
Processing bag 60 of 594
Processing bag 80 of 594
Processing bag 100 of 594
Processing bag 120 of 594
Processing bag 140 of 594
Processing bag 160 of 594
Processing bag 180 of 594
Processing bag 200 of 594
Processing bag 220 of 594
Processing bag 240 of 594
Processing bag 260 of 594
Processing bag 280 of 594
Processing bag 300 of 594
Processing bag 320 of 594
Processing bag 340 of 594
Processing bag 360 of 594
Processing bag 380 of 594
Processing bag 400 of 594
Processing bag 420 of 594
Processing bag 440 of 594
Processing bag 460 of 594
Processing bag 480 of 594
Processing bag 500 of 594
Processing bag 520 of 594
Processing bag 540 of 594
Processing bag 560 of 594
Processing bag 580 of 594


,shiny aqua,muted blue,drab gray,muted indigo,drab white,dim lavender,dotted tomato,clear orange,bright lime,striped brown,...,striped cyan,muted tomato,dotted red,dark beige,dark cyan,dull lime,light brown,vibrant magenta,shiny brown,drab purple
shiny aqua,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
muted blue,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
drab gray,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
muted indigo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
drab white,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## PART 1

In [7]:
def BagsThatCanContainABag(bagDF, bag):
    return bagDF.loc[bagDF[bag] > 0].index.values.tolist()

def CumulativeBagSearch(bagDF, wantedBag):
    cumulativeListOfBags = []
    listOfBagsToSearch = [wantedBag]
    listOfBagsThatContainSearchedBags = []

    while len(listOfBagsToSearch) != 0:
        for bagToSearch in listOfBagsToSearch:
            for containee in BagsThatCanContainABag(bagDF, bagToSearch):
                AppendToNonRepeatingList(listOfBagsThatContainSearchedBags, containee)

        for bag in listOfBagsThatContainSearchedBags:
            AppendToNonRepeatingList(cumulativeListOfBags,bag)

        listOfBagsToSearch = listOfBagsThatContainSearchedBags
        listOfBagsThatContainSearchedBags = []

    return cumulativeListOfBags

In [8]:
bagSolutionList = CumulativeBagSearch(bagData, 'shiny gold')
#print(bagSolutionList)
print(len(bagSolutionList))

265


## PART 2

In [9]:
def BagsThatABagMustContain(bagDF, bag):
    dictionary = {}
    
    for containedBag in bagDF[bagData.loc[bag] > 0].index.values.tolist():
        dictionary[containedBag] = bagData.loc[bag, containedBag]
    
    return dictionary

In [10]:
dicOfCumulativeBags = {}
dicOfBagsToSearch = {'shiny gold': 1}
bagsToSearchNextStep = {}

while len(list(dicOfBagsToSearch.keys())) != 0:
    for bagToSearch in list(dicOfBagsToSearch.keys()):
        bagsToAdd = BagsThatABagMustContain(bagData, bagToSearch)
        totalNewBagsAdded = {k: bagsToAdd.get(k,0) * dicOfBagsToSearch[bagToSearch] for k in bagsToAdd}
        bagsToSearchNextStep = {k: bagsToSearchNextStep.get(k, 0) + totalNewBagsAdded.get(k, 0) for k in set(totalNewBagsAdded) | set(bagsToSearchNextStep)}
        dicOfCumulativeBags = {k: totalNewBagsAdded.get(k, 0) + dicOfCumulativeBags.get(k, 0) for k in set(dicOfCumulativeBags) | set(bagsToSearchNextStep)}
    dicOfBagsToSearch = bagsToSearchNextStep.copy()
    bagsToSearchNextStep.clear()

print(dicOfCumulativeBags)
print(sum(dicOfCumulativeBags.values()))

{'light aqua': 425, 'muted fuchsia': 42, 'faded turquoise': 4, 'light lime': 20, 'dull magenta': 636, 'dark brown': 240, 'dull tan': 42, 'vibrant coral': 240, 'bright orange': 240, 'plaid indigo': 80, 'dark white': 21, 'shiny indigo': 2886, 'wavy coral': 1275, 'light green': 240, 'faded cyan': 168, 'light purple': 1957, 'plaid lavender': 593, 'dim teal': 42, 'dull silver': 4066, 'bright red': 960}
14177


## PART 2/ALT

In [50]:
def NestedBagSearch(bagDF, bag):
    dictionary = BagsThatABagMustContain(bagDF, bag)
    if not dictionary:
        return 1
    
    ## We begin with one bag (the searched one), then we sum all the children
    numberOfNestedBags = 1
    
    for nestedBag in dictionary:
        numberOfNestedBags += dictionary[nestedBag] * NestedBagSearch(bagDF, nestedBag)
    
    return numberOfNestedBags

In [51]:
## The function counts all bags, included the searched one. But we want to know all bags INSIDE that one!
## Ergo, minus one
NestedBagSearch(bagData, 'shiny gold') - 1

14177